In [0]:
#from google.cloud import storage
import os

# Pyspark
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

### Get the Dataset:

In [0]:
%sh pip install -q kaggle

WARNING: You are using pip version 20.2.4; however, version 21.1.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
%sh
export KAGGLE_USERNAME="calendr13"
export KAGGLE_KEY="42f8253eb72a048b22603fb48cbd2aca"
kaggle competitions download -c new-york-city-taxi-fare-prediction

0%| | 0.00/1.56G [00:00<?, ?B/s]
 0%| | 1.00M/1.56G [00:00<04:03, 6.89MB/s]
 0%| | 3.00M/1.56G [00:00<02:43, 10.2MB/s]
 0%| | 5.00M/1.56G [00:00<02:04, 13.5MB/s]
 1%| | 8.00M/1.56G [00:00<01:28, 18.8MB/s]
 1%| | 13.0M/1.56G [00:00<00:58, 28.5MB/s]
 1%|▏ | 21.0M/1.56G [00:00<00:37, 44.4MB/s]
 2%|▏ | 32.0M/1.56G [00:00<00:25, 65.0MB/s]
 2%|▏ | 39.0M/1.56G [00:00<00:24, 66.6MB/s]
 3%|▎ | 46.0M/1.56G [00:01<00:24, 67.4MB/s]
 4%|▎ | 57.0M/1.56G [00:01<00:26, 61.0MB/s]
 5%|▍ | 73.0M/1.56G [00:01<00:25, 62.2MB/s]
 5%|▌ | 83.0M/1.56G [00:01<00:22, 69.7MB/s]
 6%|▌ | 91.0M/1.56G [00:01<00:22, 70.4MB/s]
 6%|▌ | 99.0M/1.56G [00:01<00:26, 60.3MB/s]
 7%|▋ | 113M/1.56G [00:02<00:24, 63.9MB/s] 
 8%|▊ | 129M/1.56G [00:02<00:21, 71.0MB/s]
 9%|▉ | 148M/1.56G [00:02<00:15, 95.6MB/s]
 10%|▉ | 159M/1.56G [00:02<00:21, 71.4MB/s]
 11%|█ | 177M/1.56G [00:02<00:20, 73.0MB/s]
 12%|█▏ | 196M/1.56G [00:03<00:15, 94.2MB/s]
 13%|█▎ | 208M/1.56G [00:03<00:21, 66.5MB/s]
 14%|█▍ | 225M/1.56G [00:03<00:18, 77.2MB/s]
 15%|█▍ | 235M/1.56G [00:03<00:18, 79.1MB/s]
 15%|█▌ | 245M/1.56G [00:03<00:20, 69.2MB/s]
 17%|█▋ | 265M/1.56G [00:04<00:14, 93.9MB/s]
 17%|█▋ | 277M/1.56G [00:04<00:15, 90.1MB/s]
 18%|█▊ | 289M/1.56G [00:04<00:14, 96.6MB/s]
 19%|█▉ | 305M/1.56G [00:04<00:12, 111MB/s] 
 20%|█▉ | 317M/1.56G [00:04<00:12, 111MB/s]
 21%|██ | 329M/1.56G [00:04<00:17, 76.9MB/s]
 22%|██▏ | 349M/1.56G [00:04<00:12, 102MB/s] 
 23%|██▎ | 362M/1.56G [00:05<00:13, 93.5MB/s]
 24%|██▎ | 379M/1.56G [00:05<00:11, 111MB/s] 
 25%|██▍ | 392M/1.56G [00:05<00:11, 113MB/s]
 25%|██▌ | 405M/1.56G [00:05<00:11, 108MB/s]
 27%|██▋ | 424M/1.56G [00:05<00:09, 130MB/s]
 27%|██▋ | 438M/1.56G [00:05<00:15, 76.4MB/s]
 29%|██▊ | 458M/1.56G [00:06<00:12, 98.4MB/s]
 30%|██▉ | 472M/1.56G [00:06<00:14, 81.9MB/s]
 31%|███ | 488M/1.56G [00:06<00:12, 96.7MB/s]
 31%|███▏ | 501M/1.56G [00:06<00:12, 92.8MB/s]
 33%|███▎ | 520M/1.56G [00:06<00:09, 114MB/s] 
 33%|███▎ | 534M/1.56G [00:06<00:10, 109MB/s]
 34%|███▍ | 547M/1.56G [00:07<00:14, 74.9MB/s]
 35%|███▌ | 566M/1.56G [00:07<00:11, 96.2MB/s]
 37%|███▋ | 584M/1.56G [00:07<00:09, 114MB/s] 
 38%|███▊ | 599M/1.56G [00:07<00:09, 106MB/s]
 39%|███▊ | 617M/1.56G [00:07<00:08, 120MB/s]
 40%|███▉ | 631M/1.56G [00:07<00:08, 121MB/s]
 41%|████ | 649M/1.56G [00:07<00:08, 123MB/s]
 42%|████▏ | 665M/1.56G [00:08<00:10, 97.0MB/s]
 43%|████▎ | 684M/1.56G [00:08<00:08, 116MB/s] 
 44%|████▎ | 698M/1.56G [00:08<00:08, 105MB/s]
 45%|████▍ | 717M/1.56G [00:08<00:07, 125MB/s]
 46%|████▌ | 731M/1.56G [00:09<00:14, 61.4MB/s]
 47%|████▋ | 750M/1.56G [00:09<00:11, 79.8MB/s]
 48%|████▊ | 763M/1.56G [00:09<00:11, 73.3MB/s]
 49%|████▉ | 782M/1.56G [00:09<00:09, 93.2MB/s]
 50%|█████ | 801M/1.56G [00:09<00:08, 95.0MB/s]
 51%|█████▏ | 820M/1.56G [00:09<00:07, 114MB/s] 
 52%|█████▏ | 835M/1.56G [00:10<00:06, 121MB/s]
 53%|█████▎ | 850M/1.56G [00:10<00:06, 120MB/s]
 54%|█████▍ | 869M/1.56G [00:10<00:05, 138MB/s]
 55%|█████▌ | 884M/1.56G [00:10<00:09, 81.8MB/s]
 56%|█████▌ | 897M/1.56G [00:10<00:08, 90.0MB/s]
 57%|█████▋ | 911M/1.56G [00:10<00:07, 99.7MB/s]
 58%|█████▊ | 924M/1.56G [00:11<00:10, 68.5MB/s]
 59%|█████▉ | 943M/1.56G [00:11<00:07, 89.5MB/s]
 60%|█████▉ | 956M/1.56G [00:11<00:08, 78.7MB/s]
 61%|██████ | 975M/1.56G [00:11<00:06, 99.8MB/s]
 62%|██████▏ | 988M/1.56G [00:11<00:06, 92.8MB/s]
 63%|██████▎ | 0.98G/1.56G [00:11<00:05, 114MB/s]
 64%|██████▍ | 1.00G/1.56G [00:12<00:05, 110MB/s]
 65%|██████▍ | 1.01G/1.56G [00:12<00:05, 102MB/s]
 66%|██████▌ | 1.03G/1.56G [00:12<00:04, 122MB/s]
 67%|██████▋ | 1.04G/1.56G [00:12<00:05, 109MB/s]
 68%|██████▊ | 1.05G/1.56G [00:12<00:05, 92.9MB/s]
 69%|██████▉ | 1.07G/1.56G [00:12<00:04, 116MB/s] 
 70%|██████▉ | 1.09G/1.56G [00:12<00:04, 120MB/s]
 71%|███████ | 1.10G/1.56G [00:13<00:03, 138MB/s]
 72%|███████▏ | 1.12G/1.56G [00:13<00:03, 126MB/s]
 73%|███████▎ | 1.13G/1.56G [00:13<00:04, 111MB/s]
 74%|███████▎ | 1.15G/1.56G [00:13<00:03, 115MB/s]
 74%|███████▍ | 1.16G/1.56G [00:13<00:03, 114MB/s]
 76%|███████▌ | 1.18G/1.56G [00:13<00:03, 135MB/s]

In [0]:
%sh unzip new-york-city-taxi-fare-prediction.zip

Archive: new-york-city-taxi-fare-prediction.zip
 inflating: GCP-Coupons-Instructions.rtf 
 inflating: sample_submission.csv 
 inflating: test.csv 
 inflating: train.csv

In [0]:
dbutils.fs.mv("file:/databricks/driver/train.csv", "dbfs:/bdc-2020-21/datasets/NYCTaxi/train.csv")
dbutils.fs.mv("file:/databricks/driver/test.csv", "dbfs:/bdc-2020-21/datasets/NYCTaxi/test.csv")

Out[30]: True

In [0]:
%sh rm new-york-city-taxi-fare-prediction.zip
%sh rm GCP-Coupons-Instructions.rtf
%sh rm train.csv
%sh rm test.csv

rm: cannot remove 'new-york-city-taxi-fare-prediction.zip': No such file or directory
/bin/bash: line 1: fg: no job control
/bin/bash: line 2: fg: no job control
/bin/bash: line 3: fg: no job control

In [0]:
### EDA

In [0]:
from time import time

In [0]:
s = time()
train = spark.read.load("dbfs:/bdc-2020-21/datasets/NYCTaxi/train.csv", format="csv",inferSchema="true",  header="true")
print(time() - s)

257.29566526412964

In [0]:
s = time()
train.describe().show()
print(time() - s)

+-------+--------------------+------------------+--------------------+------------------+-----------------+------------------+------------------+------------------+
summary| key| fare_amount| pickup_datetime| pickup_longitude| pickup_latitude| dropoff_longitude| dropoff_latitude| passenger_count|
+-------+--------------------+------------------+--------------------+------------------+-----------------+------------------+------------------+------------------+
 count| 55423856| 55423856| 55423856| 55423856| 55423856| 55423480| 55423480| 55423856|
 mean| null|11.345045601663852| null|-72.50968444358729| 39.9197917868882|-72.51120972971809|39.920681444828844|1.6853799201556816|
 stddev| null| 20.7108321982325| null| 12.84888338140265|9.642353041994934|12.782196517830771| 9.633345796415124|1.3276643570959683|
 min|2009-01-01 00:00:...| -300.0|2009-01-01 00:00:...| -3442.059565| -3492.263768| -3442.024565| -3547.886698| 0|
 max|2015-06-30 23:59:...| 93963.36|2015-06-30 23:59:...| 3457.625683| 3408.789565| 3457.62235| 3537.132528| 208|
+-------+--------------------+------------------+--------------------+------------------+-----------------+------------------+------------------+------------------+

865.0684657096863